In [49]:
import os
import random
from concurrent.futures import ThreadPoolExecutor 
import threading
import time
from typing import Optional

import datasets
import groq
import tqdm
import torch

In [14]:
class GroqInterface:
    _client = None 

    LLAMA3_70B = "llama3-70b-8192"

    rate_lock = threading.Lock()
    error = None

    def __init__(self, model:Optional[str]=None):
        if GroqInterface._client is None:
            api_key = os.environ.get("GROQ_API_KEY")

            if api_key is None:
                raise RuntimeError("API key is not in the environment variables ('GROQ_API_KEY' variable is not set).")

            GroqInterface._client = groq.Groq(api_key=api_key)

        if model is None:
            model = GroqInterface.LLAMA3_70B
        self._model = model

    def __call__(self, prompt:str):
        done = False
        while not done:

            try:
                GroqInterface.rate_lock.acquire()
                GroqInterface.rate_lock.release()

                chat_completion = GroqInterface._client.chat.completions.create(
                        messages=[
                            {
                                "role": "user",
                                "content": prompt,
                            }
                        ],
                        model=self._model,
                    )
                
                done = True
            except groq.RateLimitError as exception:
                GroqInterface.error = exception
                if not GroqInterface.rate_lock.locked():
                    GroqInterface.rate_lock.acquire()
                    time.sleep(2)
                    GroqInterface.rate_lock.release()

        return chat_completion.choices[0].message.content

In [15]:
groq_interface = GroqInterface()

In [16]:
groq_interface("Hi!")

"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [17]:
POSITIVE = 1
NEGATIVE = 0

In [18]:
class GroqSentimentInterface(GroqInterface):

    def __call__(self, prompt: str):
        response = super().__call__(prompt)
        response = response.lower()

        if "positive" in response and "negative" not in response:
            return POSITIVE
        if "negative" in response and "positive" not in response:
            return NEGATIVE
        
        return random.choice([POSITIVE, NEGATIVE])

In [19]:
groq_sentiment = GroqSentimentInterface()

## IMDB Prompt Engineering

In [10]:
executor = ThreadPoolExecutor(max_workers=2)

trainbase_future = executor.submit(datasets.load_dataset, "imdb", split="train")
test_future = executor.submit(datasets.load_dataset, "imdb", split='test')

trainbase_dataset = trainbase_future.result()
testbase_dataset = test_future.result()

train_val_dataset = trainbase_dataset.train_test_split(test_size=100, shuffle=True, seed=78)
discard_test_dataset = testbase_dataset.train_test_split(test_size=100, shuffle=True, seed=78)

train_dataset = train_val_dataset["train"]
val_dataset = train_val_dataset["test"]
test_dataset = discard_test_dataset["test"]

In [11]:
len(train_dataset), len(val_dataset), len(test_dataset)

(24900, 100, 100)

## Zero-shot

In [20]:
base_prompt_zero = '''Classify if the movie review is POSITIVE or NEGATIVE: 
                Review:
                {review}

                Sentiment:
                POSITIVE OR NEGATIVE: 
                '''

In [44]:
prompt = base_prompt_zero.format(review=train_dataset[-1]["text"])

groq_sentiment(prompt), train_dataset[-1]["label"]

(1, 1)

In [22]:
def evaluate_zero(text, label):
    prompt = base_prompt_zero.format(review=text)
    result = groq_sentiment(prompt)

    return result == label

In [23]:
executor = ThreadPoolExecutor(max_workers=4) #More workers -> More RateLimit exceptions

futures = []
for data in val_dataset:
    future = executor.submit(evaluate_zero, **data)
    futures.append(future)

correct_zero = 0
for future in tqdm.tqdm(futures):
    correct_zero += future.result()

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [07:30<00:00,  4.50s/it] 


In [29]:
accuracy_zero = correct_zero/len(val_dataset)
print(f"Acurácia - Zero-shot - Validação: {accuracy_zero*100}%")

Acurácia - Zero-shot - Validação: 88.0%


Acurácia - Zero-shot - Validação: 88.0%


## Few-shot

In [34]:
raw_prompt_few = '''Classify if the movie review is positive or negative: 
                Review:
                Movie review

                Sentiment:
                ONLY POSITIVE OR NEGATIVE

                Classify if this movie review is positive or negative:
                Review:
                {example1}

                Sentiment:
                {response1}

                Classify if this movie review is positive or negative:
                Review:
                {example2}

                Sentiment:
                {response2}

                Classify if this movie review is positive or negative:
                Review:
                {{review}}
                
                Sentiment:
                
                '''

In [38]:
positive_example = None
negative_example = None

i = 0
while positive_example is None or negative_example is None:
    if train_dataset[i]["label"] == POSITIVE:
        positive_example = train_dataset[i]
    else:
        negative_example = train_dataset[i]
    
    i += 1

In [39]:
base_prompt_few = raw_prompt_few.format(example1=positive_example["text"], response1="POSITIVE", 
                                        example2=negative_example["text"], response2="NEGATIVE")

print(base_prompt_few)

Classify if the movie review is positive or negative: 
                Review:
                Movie review

                Sentiment:
                ONLY POSITIVE OR NEGATIVE

                Classify if this movie review is positive or negative:
                Review:
                but I want to say I cannot agree more with Moira.<br /><br />What a wonderful film.<br /><br />I was thinking about it just this morning, wanting to give advice to some dopey sod who'd lost money on his debit card through fraud, and wanted to say 'Keep thy money in thine pocket' and realised I was talking like James Mason.<br /><br />Even tho he didn't say those words, I still think he would! I've never forgotten 'Are ye carrying?' in his reconciliation with his son, Hywel Bennet: 'Always have money in thine pocket!' Good advice.<br /><br />Not enough kids have fathers with such unforgiving but well-meant attitudes any more. Or any father at all.<br /><br />It would be a good thing for us to reinstate

In [42]:
prompt = base_prompt_few.format(review=train_dataset[-1]["text"])

groq_sentiment(prompt), train_dataset[-1]["label"]

(1, 1)

In [45]:
def evaluate_few(text, label):
    prompt = base_prompt_few.format(review=text)
    result = groq_sentiment(prompt)

    return result == label

In [46]:
evaluate_few(**train_dataset[-1])

True

In [47]:
executor = ThreadPoolExecutor(max_workers=4) #More workers -> More RateLimit exceptions

futures = []
for data in val_dataset:
    future = executor.submit(evaluate_few, **data)
    futures.append(future)

correct_few = 0
for future in tqdm.tqdm(futures):
    correct_few += future.result()

100%|██████████| 100/100 [10:14<00:00,  6.15s/it]


In [48]:
accuracy_few = correct_few/len(val_dataset)
print(f"Acurácia - Few-shot - Validação: {accuracy_few*100}%")

Acurácia - Few-shot - Validação: 98.0%


## Prompt Dinâmico

In [ ]:
bert_tokenizer = torch.hub.load('huggingface/pytorch-transformers', 'tokenizer', 'bert-base-cased')
bert = torch.hub.load('huggingface/pytorch-transformers', 'model', 'bert-base-cased')